In [1]:
# import numpy as np

# # !/usr/bin/env python3
# # -*- coding: utf-8 -*-
# """
# Created on 20181219

# @author: zhangji

# Trajection of a ellipse, Jeffery equation. 
# """

# %pylab inline
# pylab.rcParams['figure.figsize'] = (25, 11)
# fontsize = 40

# import numpy as np
# import scipy as sp
# from scipy.optimize import leastsq, curve_fit
# from scipy import interpolate
# from scipy.interpolate import interp1d
# from scipy.io import loadmat, savemat
# # import scipy.misc

# import matplotlib
# from matplotlib import pyplot as plt
# from matplotlib import animation, rc
# import matplotlib.ticker as mtick
# from mpl_toolkits.axes_grid1.inset_locator import inset_axes, zoomed_inset_axes
# from mpl_toolkits.mplot3d import Axes3D, axes3d

# from sympy import symbols, simplify, series, exp
# from sympy.matrices import Matrix
# from sympy.solvers import solve

# from IPython.display import display, HTML
# from tqdm import tqdm_notebook as tqdm
# import pandas as pd
# import re
# from scanf import scanf
# import os
# import glob

# from codeStore import support_fun as spf
# from src.support_class import *
# from src import stokes_flow as sf

# rc('animation', html='html5')
# PWD = os.getcwd()
# font = {'size': 20}
# matplotlib.rc('font', **font)

from tqdm import tqdm_notebook
import os
import glob
import natsort 
import numpy as np
import scipy as sp
from scipy.optimize import leastsq, curve_fit
from scipy import interpolate, integrate
from scipy import spatial, signal
# from scipy.interpolate import interp1d
from scipy.io import loadmat, savemat
# import scipy.misc
import importlib
from IPython.display import display, HTML
import pandas as pd
import pickle
import re
from scanf import scanf

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.colors as colors
from matplotlib import animation, rc
import matplotlib.ticker as mtick
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, zoomed_inset_axes
from mpl_toolkits.mplot3d import Axes3D, axes3d
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib import cm

from tqdm import tqdm, tqdm_notebook
from time import time
from src.support_class import *
from src import jeffery_model as jm
from codeStore import support_fun as spf
from codeStore import support_fun_table as spf_tb
%matplotlib inline

rc('animation', html='html5')
fontsize = 40
PWD = os.getcwd()
np.set_printoptions(linewidth=120, precision=5)

In [91]:
# t_u_node = np.arange(3)
# t_u_node = t_u_node / np.linalg.norm(t_u_node) * Ra
# f_nodes = np.arange(12).reshape(4, 3) ** 2
# Ra = 1

Ra = 1
nu, nf = 1, 6
uth, uph = (np.random.sample((nu, 2)) * (1, 2)).T * np.pi
uR = Ra
t_u_node = uR * np.hstack((np.sin(uth) * np.cos(uph), np.sin(uth) * np.sin(uph), np.cos(uth)))
fth, fph = (np.random.sample((nf, 2)) * (1, 2)).T * np.pi
fR = np.random.sample(nf) + 1e-3
fR = fR / fR.min() + 1e-3
f_nodes = (fR * np.array((np.sin(fth) * np.cos(fph), np.sin(fth) * np.sin(fph), np.cos(fth)))).T

img_f_nodes = (f_nodes.T * (Ra / np.linalg.norm(f_nodes, axis=-1)) ** 2).T
dxi = (t_u_node - f_nodes).T
disx = dxi[0]
disy = dxi[1]
disz = dxi[2]
r2sk = disx ** 2 + disy ** 2 + disz ** 2
rsk = r2sk ** 0.5
r3sk = r2sk ** 1.5

H1sk = 1.0 / r3sk
MSxx = (r2sk + disx ** 2) * H1sk
MSyx = disy * disx * H1sk
MSxy = MSyx
MSyy = (r2sk + disy ** 2) * H1sk
MSxz = disx * disz * H1sk
MSyz = disy * disz * H1sk
MSzx = MSxz
MSzy = MSyz
MSzz = (r2sk + disz ** 2) * H1sk

img_dxi = (t_u_node - img_f_nodes).T
disxi = img_dxi[0]
disyi = img_dxi[1]
diszi = img_dxi[2]
r2ski = disxi ** 2 + disyi ** 2 + diszi ** 2
rski = r2ski ** 0.5
r3ski = r2ski ** 1.5
r5ski = r2ski ** 2.5
H1ski = 1 / rski
H2ski = 1 / r2ski
H3ski = 1 / r3ski
H5ski = 1 / r5ski

Xf = np.linalg.norm(f_nodes, axis=1)
Xfi = np.linalg.norm(img_f_nodes, axis=1)
xf = np.linalg.norm(t_u_node)
Dx1 = np.sum(img_dxi.T * img_f_nodes, axis=1)
Dx2 = np.sum(t_u_node * img_f_nodes, axis=1)

A = 0.5 * (Xf ** 2 - Ra ** 2) / Xf ** 3
B = r2ski * (rski - Xfi) * Xfi
C = 3 * Ra / Xfi
Det = 1 / (Xfi * (Xfi * rski + Dx2 - Xfi ** 2))
E = 1 / (xf * Xfi * (xf * Xfi + Dx2))

A1 = (Xf ** 2 - Ra ** 2) / Xf
A2 = xf ** 2 - Ra ** 2

In [92]:
Pxx = A * (-3 * f_nodes[:, 0] * disxi * H3ski / Ra + Ra * H3ski - 3 * Ra * disxi ** 2 * H5ski
           - 2 * f_nodes[:, 0] * img_f_nodes[:, 0] * H3ski / Ra
           + 6 * f_nodes[:, 0] * H5ski * disxi * Dx1 / Ra
           + C * (img_f_nodes[:, 0] * disxi * r2ski + disxi ** 2 * Xfi ** 2 + B)
           * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (Xfi * disxi + img_f_nodes[:, 0] * rski)
           * (img_f_nodes[:, 0] * r2ski - disxi * Xfi ** 2 + (t_u_node[0] - 2 * img_f_nodes[:, 0])
              * rski * Xfi) - C * E * (t_u_node[0] * img_f_nodes[:, 0] + xf * Xfi)
           + C * E ** 2 * (Xfi * t_u_node[0] + xf * img_f_nodes[:, 0]) * (
                   Xfi * t_u_node[0] + xf * img_f_nodes[:, 0]) * xf * Xfi)
Pyy = A * (-3 * f_nodes[:, 1] * disyi * H3ski / Ra + Ra * H3ski - 3 * Ra * disyi ** 2 * H5ski
           - 2 * f_nodes[:, 1] * img_f_nodes[:, 1] * H3ski / Ra
           + 6 * f_nodes[:, 1] * H5ski * disyi * Dx1 / Ra
           + C * (img_f_nodes[:, 1] * disyi * r2ski + disyi ** 2 * Xfi ** 2 + B)
           * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (Xfi * disyi + img_f_nodes[:, 1] * rski)
           * (img_f_nodes[:, 1] * r2ski - disyi * Xfi ** 2
              + (t_u_node[1] - 2 * img_f_nodes[:, 1]) * rski * Xfi) - C * E * (
                   t_u_node[1] * img_f_nodes[:, 1] + xf * Xfi) + C * E ** 2 * (
                   Xfi * t_u_node[1] + xf * img_f_nodes[:, 1]) * (
                   Xfi * t_u_node[1] + xf * img_f_nodes[:, 1]) * xf * Xfi)
Pzz = A * (-3 * f_nodes[:,
                2] * diszi * H3ski / Ra + Ra * H3ski - 3 * Ra * diszi ** 2 * H5ski - 2 * f_nodes[:,
                                                                                         2] * img_f_nodes[
                                                                                              :,
                                                                                              2] * H3ski / Ra + 6 * f_nodes[
                                                                                                                    :,
                                                                                                                    2] * H5ski * diszi * Dx1 / Ra + C * (
                   img_f_nodes[:,
                   2] * diszi * r2ski + diszi ** 2 * Xfi ** 2 + B) * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (
                   Xfi * diszi + img_f_nodes[:, 2] * rski) * (
                   img_f_nodes[:, 2] * r2ski - diszi * Xfi ** 2 + (
                   t_u_node[2] - 2 * img_f_nodes[:, 2]) * rski * Xfi) - C * E * (
                   t_u_node[2] * img_f_nodes[:, 2] + xf * Xfi) + C * E ** 2 * (
                   Xfi * t_u_node[2] + xf * img_f_nodes[:, 2]) * (
                   Xfi * t_u_node[2] + xf * img_f_nodes[:, 2]) * xf * Xfi)

Pxy = A * (-3 * f_nodes[:, 1] * disxi * H3ski / Ra - 3 * Ra * disxi * disyi * H5ski - 2 * f_nodes[:,
                                                                                         1] * img_f_nodes[
                                                                                              :,
                                                                                              0] * H3ski / Ra + 6 * f_nodes[
                                                                                                                    :,
                                                                                                                    1] * H5ski * disxi * Dx1 / Ra + C * (
                   img_f_nodes[:,
                   1] * disxi * r2ski + disxi * disyi * Xfi ** 2) * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (
                   Xfi * disxi + img_f_nodes[:, 0] * rski) * (
                   img_f_nodes[:, 1] * r2ski - disyi * Xfi ** 2 + (
                   t_u_node[1] - 2 * img_f_nodes[:, 1]) * rski * Xfi) - C * E * (
                   t_u_node[0] * img_f_nodes[:, 1]) + C * E ** 2 * (
                   Xfi * t_u_node[0] + xf * img_f_nodes[:, 0]) * (
                   Xfi * t_u_node[1] + xf * img_f_nodes[:, 1]) * xf * Xfi)
Pyx = A * (-3 * f_nodes[:, 0] * disyi * H3ski / Ra - 3 * Ra * disxi * disyi * H5ski - 2 * f_nodes[:,
                                                                                         1] * img_f_nodes[
                                                                                              :,
                                                                                              0] * H3ski / Ra + 6 * f_nodes[
                                                                                                                    :,
                                                                                                                    0] * H5ski * disyi * Dx1 / Ra + C * (
                   img_f_nodes[:,
                   0] * disyi * r2ski + disxi * disyi * Xfi ** 2) * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (
                   Xfi * disyi + img_f_nodes[:, 1] * rski) * (
                   img_f_nodes[:, 0] * r2ski - disxi * Xfi ** 2 + (
                   t_u_node[0] - 2 * img_f_nodes[:, 0]) * rski * Xfi) - C * E * (
                   t_u_node[1] * img_f_nodes[:, 0]) + C * E ** 2 * (
                   Xfi * t_u_node[0] + xf * img_f_nodes[:, 0]) * (
                   Xfi * t_u_node[1] + xf * img_f_nodes[:, 1]) * xf * Xfi)

Pxz = A * (-3 * f_nodes[:, 2] * disxi * H3ski / Ra - 3 * Ra * disxi * diszi * H5ski - 2 * f_nodes[:,
                                                                                         2] * img_f_nodes[
                                                                                              :,
                                                                                              0] * H3ski / Ra + 6 * f_nodes[
                                                                                                                    :,
                                                                                                                    2] * H5ski * disxi * Dx1 / Ra + C * (
                   img_f_nodes[:,
                   2] * disxi * r2ski + disxi * diszi * Xfi ** 2) * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (
                   Xfi * disxi + img_f_nodes[:, 0] * rski) * (
                   img_f_nodes[:, 2] * r2ski - diszi * Xfi ** 2 + (
                   t_u_node[2] - 2 * img_f_nodes[:, 2]) * rski * Xfi) - C * E * (
                   t_u_node[0] * img_f_nodes[:, 2]) + C * E ** 2 * (
                   Xfi * t_u_node[0] + xf * img_f_nodes[:, 0]) * (
                   Xfi * t_u_node[2] + xf * img_f_nodes[:, 2]) * xf * Xfi)
Pzx = A * (-3 * f_nodes[:, 0] * diszi * H3ski / Ra - 3 * Ra * disxi * diszi * H5ski - 2 * f_nodes[:,
                                                                                         2] * img_f_nodes[
                                                                                              :,
                                                                                              0] * H3ski / Ra + 6 * f_nodes[
                                                                                                                    :,
                                                                                                                    0] * H5ski * diszi * Dx1 / Ra + C * (
                   img_f_nodes[:,
                   0] * diszi * r2ski + disxi * diszi * Xfi ** 2) * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (
                   Xfi * diszi + img_f_nodes[:, 2] * rski) * (
                   img_f_nodes[:, 0] * r2ski - disxi * Xfi ** 2 + (
                   t_u_node[0] - 2 * img_f_nodes[:, 0]) * rski * Xfi) - C * E * (
                   t_u_node[2] * img_f_nodes[:, 0]) + C * E ** 2 * (
                   Xfi * t_u_node[0] + xf * img_f_nodes[:, 0]) * (
                   Xfi * t_u_node[2] + xf * img_f_nodes[:, 2]) * xf * Xfi)

Pyz = A * (-3 * f_nodes[:, 2] * disyi * H3ski / Ra - 3 * Ra * diszi * disyi * H5ski - 2 * f_nodes[:,
                                                                                         1] * img_f_nodes[
                                                                                              :,
                                                                                              2] * H3ski / Ra + 6 * f_nodes[
                                                                                                                    :,
                                                                                                                    2] * H5ski * disyi * Dx1 / Ra + C * (
                   img_f_nodes[:,
                   2] * disyi * r2ski + diszi * disyi * Xfi ** 2) * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (
                   Xfi * disyi + img_f_nodes[:, 1] * rski) * (
                   img_f_nodes[:, 2] * r2ski - diszi * Xfi ** 2 + (
                   t_u_node[2] - 2 * img_f_nodes[:, 2]) * rski * Xfi) - C * E * (
                   t_u_node[1] * img_f_nodes[:, 2]) + C * E ** 2 * (
                   Xfi * t_u_node[2] + xf * img_f_nodes[:, 2]) * (
                   Xfi * t_u_node[1] + xf * img_f_nodes[:, 1]) * xf * Xfi)
Pzy = A * (-3 * f_nodes[:, 1] * diszi * H3ski / Ra - 3 * Ra * disyi * diszi * H5ski - 2 * f_nodes[:,
                                                                                         2] * img_f_nodes[
                                                                                              :,
                                                                                              1] * H3ski / Ra + 6 * f_nodes[
                                                                                                                    :,
                                                                                                                    1] * H5ski * diszi * Dx1 / Ra + C * (
                   img_f_nodes[:,
                   1] * diszi * r2ski + disyi * diszi * Xfi ** 2) * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (
                   Xfi * diszi + img_f_nodes[:, 2] * rski) * (
                   img_f_nodes[:, 1] * r2ski - disyi * Xfi ** 2 + (
                   t_u_node[1] - 2 * img_f_nodes[:, 1]) * rski * Xfi) - C * E * (
                   t_u_node[2] * img_f_nodes[:, 1]) + C * E ** 2 * (
                   Xfi * t_u_node[1] + xf * img_f_nodes[:, 1]) * (
                   Xfi * t_u_node[2] + xf * img_f_nodes[:, 2]) * xf * Xfi)

Mxx = MSxx - Ra * H1ski / Xf - Ra ** 3 * disxi * disxi * H3ski / Xf ** 3 - A1 * (
        img_f_nodes[:, 0] * img_f_nodes[:, 0] * H1ski / Ra ** 3 - Ra * H3ski * (
        img_f_nodes[:, 0] * disxi + img_f_nodes[:, 0] * disxi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                             0] * img_f_nodes[:,
                                                                                  0] * Dx1 * H3ski / Ra ** 3) - A2 * Pxx
Myy = MSyy - Ra * H1ski / Xf - Ra ** 3 * disyi * disyi * H3ski / Xf ** 3 - A1 * (
        img_f_nodes[:, 1] * img_f_nodes[:, 1] * H1ski / Ra ** 3 - Ra * H3ski * (
        img_f_nodes[:, 1] * disyi + img_f_nodes[:, 1] * disyi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                             1] * img_f_nodes[:,
                                                                                  1] * Dx1 * H3ski / Ra ** 3) - A2 * Pyy
Mzz = MSzz - Ra * H1ski / Xf - Ra ** 3 * diszi * diszi * H3ski / Xf ** 3 - A1 * (
        img_f_nodes[:, 2] * img_f_nodes[:, 2] * H1ski / Ra ** 3 - Ra * H3ski * (
        img_f_nodes[:, 2] * diszi + img_f_nodes[:, 2] * diszi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                             2] * img_f_nodes[:,
                                                                                  2] * Dx1 * H3ski / Ra ** 3) - A2 * Pzz

Mxy = MSxy - Ra ** 3 * disxi * disyi * H3ski / Xf ** 3 - A1 * (
        img_f_nodes[:, 0] * img_f_nodes[:, 1] * H1ski / Ra ** 3 - Ra * H3ski * (
        img_f_nodes[:, 0] * disyi + img_f_nodes[:, 1] * disxi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                             1] * img_f_nodes[:,
                                                                                  0] * Dx1 * H3ski / Ra ** 3) - A2 * Pxy
Mxz = MSxz - Ra ** 3 * disxi * diszi * H3ski / Xf ** 3 - A1 * (
        img_f_nodes[:, 0] * img_f_nodes[:, 2] * H1ski / Ra ** 3 - Ra * H3ski * (
        img_f_nodes[:, 0] * diszi + img_f_nodes[:, 2] * disxi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                             2] * img_f_nodes[:,
                                                                                  0] * Dx1 * H3ski / Ra ** 3) - A2 * Pxz
Myz = MSyz - Ra ** 3 * disyi * diszi * H3ski / Xf ** 3 - A1 * (
        img_f_nodes[:, 2] * img_f_nodes[:, 1] * H1ski / Ra ** 3 - Ra * H3ski * (
        img_f_nodes[:, 2] * disyi + img_f_nodes[:, 1] * diszi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                             1] * img_f_nodes[:,
                                                                                  2] * Dx1 * H3ski / Ra ** 3) - A2 * Pyz

Myx = MSyx - Ra ** 3 * disxi * disyi * H3ski / Xf ** 3 - A1 * (
        img_f_nodes[:, 0] * img_f_nodes[:, 1] * H1ski / Ra ** 3 - Ra * H3ski * (
        img_f_nodes[:, 0] * disyi + img_f_nodes[:, 1] * disxi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                             1] * img_f_nodes[:,
                                                                                  0] * Dx1 * H3ski / Ra ** 3) - A2 * Pyx
Mzx = MSzx - Ra ** 3 * disxi * diszi * H3ski / Xf ** 3 - A1 * (
        img_f_nodes[:, 0] * img_f_nodes[:, 2] * H1ski / Ra ** 3 - Ra * H3ski * (
        img_f_nodes[:, 0] * diszi + img_f_nodes[:, 2] * disxi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                             2] * img_f_nodes[:,
                                                                                  0] * Dx1 * H3ski / Ra ** 3) - A2 * Pzx
Mzy = MSzy - Ra ** 3 * disyi * diszi * H3ski / Xf ** 3 - A1 * (
        img_f_nodes[:, 2] * img_f_nodes[:, 1] * H1ski / Ra ** 3 - Ra * H3ski * (
        img_f_nodes[:, 2] * disyi + img_f_nodes[:, 1] * diszi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                             1] * img_f_nodes[:,
                                                                                  2] * Dx1 * H3ski / Ra ** 3) - A2 * Pzy


In [75]:
Mzy

array([1.86742676e-17, 2.32770712e-18, 5.07546357e-19, 1.54366602e-18])

In [95]:
def pf_sphere_image_petsc_mij(t_u_node: np.ndarray,  # velocity node
                              f_nodes: np.ndarray,  # force nodes
                              i0, **kwargs):
    Ra = kwargs['pf_sphere_image_Ra']
    img_f_nodes = (f_nodes.T * (Ra / np.linalg.norm(f_nodes, axis=-1)) ** 2).T
    dxi = (t_u_node - f_nodes).T
    disx = dxi[0]
    disy = dxi[1]
    disz = dxi[2]
    r2sk = disx ** 2 + disy ** 2 + disz ** 2
    rsk = r2sk ** 0.5
    r3sk = r2sk ** 1.5

    H1sk = 1.0 / r3sk
    MSxx = (r2sk + disx ** 2) * H1sk
    MSyx = disy * disx * H1sk
    MSxy = MSyx
    MSyy = (r2sk + disy ** 2) * H1sk
    MSxz = disx * disz * H1sk
    MSyz = disy * disz * H1sk
    MSzx = MSxz
    MSzy = MSyz
    MSzz = (r2sk + disz ** 2) * H1sk

    img_dxi = (t_u_node - img_f_nodes).T
    disxi = img_dxi[0]
    disyi = img_dxi[1]
    diszi = img_dxi[2]
    r2ski = disxi ** 2 + disyi ** 2 + diszi ** 2
    rski = r2ski ** 0.5
    r3ski = r2ski ** 1.5
    r5ski = r2ski ** 2.5
    H1ski = 1 / rski
    H2ski = 1 / r2ski
    H3ski = 1 / r3ski
    H5ski = 1 / r5ski

    Xf = np.linalg.norm(f_nodes, axis=1)
    Xfi = np.linalg.norm(img_f_nodes, axis=1)
    xf = np.linalg.norm(t_u_node)
    Dx1 = np.sum(img_dxi.T * img_f_nodes, axis=1)
    Dx2 = np.sum(t_u_node * img_f_nodes, axis=1)

    A = 0.5 * (Xf ** 2 - Ra ** 2) / Xf ** 3
    B = r2ski * (rski - Xfi) * Xfi
    C = 3 * Ra / Xfi
    Det = 1 / (Xfi * (Xfi * rski + Dx2 - Xfi ** 2))
    E = 1 / (xf * Xfi * (xf * Xfi + Dx2))
    A1 = (Xf ** 2 - Ra ** 2) / Xf
    A2 = xf ** 2 - Ra ** 2.5
    
    Pxx = A * (-3 * f_nodes[:, 0] * disxi * H3ski / Ra + Ra * H3ski - 3 * Ra * disxi ** 2 * H5ski
               - 2 * f_nodes[:, 0] * img_f_nodes[:, 0] * H3ski / Ra
               + 6 * f_nodes[:, 0] * H5ski * disxi * Dx1 / Ra
               + C * (img_f_nodes[:, 0] * disxi * r2ski + disxi ** 2 * Xfi ** 2 + B)
               * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (Xfi * disxi + img_f_nodes[:, 0] * rski)
               * (img_f_nodes[:, 0] * r2ski - disxi * Xfi ** 2 + (t_u_node[0] - 2 * img_f_nodes[:, 0])
                  * rski * Xfi) - C * E * (t_u_node[0] * img_f_nodes[:, 0] + xf * Xfi)
               + C * E ** 2 * (Xfi * t_u_node[0] + xf * img_f_nodes[:, 0]) * (
                       Xfi * t_u_node[0] + xf * img_f_nodes[:, 0]) * xf * Xfi)
    Pyy = A * (-3 * f_nodes[:, 1] * disyi * H3ski / Ra + Ra * H3ski - 3 * Ra * disyi ** 2 * H5ski
               - 2 * f_nodes[:, 1] * img_f_nodes[:, 1] * H3ski / Ra
               + 6 * f_nodes[:, 1] * H5ski * disyi * Dx1 / Ra
               + C * (img_f_nodes[:, 1] * disyi * r2ski + disyi ** 2 * Xfi ** 2 + B)
               * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (Xfi * disyi + img_f_nodes[:, 1] * rski)
               * (img_f_nodes[:, 1] * r2ski - disyi * Xfi ** 2
                  + (t_u_node[1] - 2 * img_f_nodes[:, 1]) * rski * Xfi) - C * E * (
                       t_u_node[1] * img_f_nodes[:, 1] + xf * Xfi) + C * E ** 2 * (
                       Xfi * t_u_node[1] + xf * img_f_nodes[:, 1]) * (
                       Xfi * t_u_node[1] + xf * img_f_nodes[:, 1]) * xf * Xfi)
    Pzz = A * (-3 * f_nodes[:,
                    2] * diszi * H3ski / Ra + Ra * H3ski - 3 * Ra * diszi ** 2 * H5ski - 2 * f_nodes[:,
                                                                                             2] * img_f_nodes[
                                                                                                  :,
                                                                                                  2] * H3ski / Ra + 6 * f_nodes[
                                                                                                                        :,
                                                                                                                        2] * H5ski * diszi * Dx1 / Ra + C * (
                       img_f_nodes[:,
                       2] * diszi * r2ski + diszi ** 2 * Xfi ** 2 + B) * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (
                       Xfi * diszi + img_f_nodes[:, 2] * rski) * (
                       img_f_nodes[:, 2] * r2ski - diszi * Xfi ** 2 + (
                       t_u_node[2] - 2 * img_f_nodes[:, 2]) * rski * Xfi) - C * E * (
                       t_u_node[2] * img_f_nodes[:, 2] + xf * Xfi) + C * E ** 2 * (
                       Xfi * t_u_node[2] + xf * img_f_nodes[:, 2]) * (
                       Xfi * t_u_node[2] + xf * img_f_nodes[:, 2]) * xf * Xfi)

    Pxy = A * (-3 * f_nodes[:, 1] * disxi * H3ski / Ra - 3 * Ra * disxi * disyi * H5ski - 2 * f_nodes[:,
                                                                                             1] * img_f_nodes[
                                                                                                  :,
                                                                                                  0] * H3ski / Ra + 6 * f_nodes[
                                                                                                                        :,
                                                                                                                        1] * H5ski * disxi * Dx1 / Ra + C * (
                       img_f_nodes[:,
                       1] * disxi * r2ski + disxi * disyi * Xfi ** 2) * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (
                       Xfi * disxi + img_f_nodes[:, 0] * rski) * (
                       img_f_nodes[:, 1] * r2ski - disyi * Xfi ** 2 + (
                       t_u_node[1] - 2 * img_f_nodes[:, 1]) * rski * Xfi) - C * E * (
                       t_u_node[0] * img_f_nodes[:, 1]) + C * E ** 2 * (
                       Xfi * t_u_node[0] + xf * img_f_nodes[:, 0]) * (
                       Xfi * t_u_node[1] + xf * img_f_nodes[:, 1]) * xf * Xfi)
    Pyx = A * (-3 * f_nodes[:, 0] * disyi * H3ski / Ra - 3 * Ra * disxi * disyi * H5ski - 2 * f_nodes[:,
                                                                                             1] * img_f_nodes[
                                                                                                  :,
                                                                                                  0] * H3ski / Ra + 6 * f_nodes[
                                                                                                                        :,
                                                                                                                        0] * H5ski * disyi * Dx1 / Ra + C * (
                       img_f_nodes[:,
                       0] * disyi * r2ski + disxi * disyi * Xfi ** 2) * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (
                       Xfi * disyi + img_f_nodes[:, 1] * rski) * (
                       img_f_nodes[:, 0] * r2ski - disxi * Xfi ** 2 + (
                       t_u_node[0] - 2 * img_f_nodes[:, 0]) * rski * Xfi) - C * E * (
                       t_u_node[1] * img_f_nodes[:, 0]) + C * E ** 2 * (
                       Xfi * t_u_node[0] + xf * img_f_nodes[:, 0]) * (
                       Xfi * t_u_node[1] + xf * img_f_nodes[:, 1]) * xf * Xfi)

    Pxz = A * (-3 * f_nodes[:, 2] * disxi * H3ski / Ra - 3 * Ra * disxi * diszi * H5ski - 2 * f_nodes[:,
                                                                                             2] * img_f_nodes[
                                                                                                  :,
                                                                                                  0] * H3ski / Ra + 6 * f_nodes[
                                                                                                                        :,
                                                                                                                        2] * H5ski * disxi * Dx1 / Ra + C * (
                       img_f_nodes[:,
                       2] * disxi * r2ski + disxi * diszi * Xfi ** 2) * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (
                       Xfi * disxi + img_f_nodes[:, 0] * rski) * (
                       img_f_nodes[:, 2] * r2ski - diszi * Xfi ** 2 + (
                       t_u_node[2] - 2 * img_f_nodes[:, 2]) * rski * Xfi) - C * E * (
                       t_u_node[0] * img_f_nodes[:, 2]) + C * E ** 2 * (
                       Xfi * t_u_node[0] + xf * img_f_nodes[:, 0]) * (
                       Xfi * t_u_node[2] + xf * img_f_nodes[:, 2]) * xf * Xfi)
    Pzx = A * (-3 * f_nodes[:, 0] * diszi * H3ski / Ra - 3 * Ra * disxi * diszi * H5ski - 2 * f_nodes[:,
                                                                                             2] * img_f_nodes[
                                                                                                  :,
                                                                                                  0] * H3ski / Ra + 6 * f_nodes[
                                                                                                                        :,
                                                                                                                        0] * H5ski * diszi * Dx1 / Ra + C * (
                       img_f_nodes[:,
                       0] * diszi * r2ski + disxi * diszi * Xfi ** 2) * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (
                       Xfi * diszi + img_f_nodes[:, 2] * rski) * (
                       img_f_nodes[:, 0] * r2ski - disxi * Xfi ** 2 + (
                       t_u_node[0] - 2 * img_f_nodes[:, 0]) * rski * Xfi) - C * E * (
                       t_u_node[2] * img_f_nodes[:, 0]) + C * E ** 2 * (
                       Xfi * t_u_node[0] + xf * img_f_nodes[:, 0]) * (
                       Xfi * t_u_node[2] + xf * img_f_nodes[:, 2]) * xf * Xfi)

    Pyz = A * (-3 * f_nodes[:, 2] * disyi * H3ski / Ra - 3 * Ra * diszi * disyi * H5ski - 2 * f_nodes[:,
                                                                                             1] * img_f_nodes[
                                                                                                  :,
                                                                                                  2] * H3ski / Ra + 6 * f_nodes[
                                                                                                                        :,
                                                                                                                        2] * H5ski * disyi * Dx1 / Ra + C * (
                       img_f_nodes[:,
                       2] * disyi * r2ski + diszi * disyi * Xfi ** 2) * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (
                       Xfi * disyi + img_f_nodes[:, 1] * rski) * (
                       img_f_nodes[:, 2] * r2ski - diszi * Xfi ** 2 + (
                       t_u_node[2] - 2 * img_f_nodes[:, 2]) * rski * Xfi) - C * E * (
                       t_u_node[1] * img_f_nodes[:, 2]) + C * E ** 2 * (
                       Xfi * t_u_node[2] + xf * img_f_nodes[:, 2]) * (
                       Xfi * t_u_node[1] + xf * img_f_nodes[:, 1]) * xf * Xfi)
    Pzy = A * (-3 * f_nodes[:, 1] * diszi * H3ski / Ra - 3 * Ra * disyi * diszi * H5ski - 2 * f_nodes[:,
                                                                                             2] * img_f_nodes[
                                                                                                  :,
                                                                                                  1] * H3ski / Ra + 6 * f_nodes[
                                                                                                                        :,
                                                                                                                        1] * H5ski * diszi * Dx1 / Ra + C * (
                       img_f_nodes[:,
                       1] * diszi * r2ski + disyi * diszi * Xfi ** 2) * H3ski * Det - C * H2ski * Det ** 2 * Xfi * (
                       Xfi * diszi + img_f_nodes[:, 2] * rski) * (
                       img_f_nodes[:, 1] * r2ski - disyi * Xfi ** 2 + (
                       t_u_node[1] - 2 * img_f_nodes[:, 1]) * rski * Xfi) - C * E * (
                       t_u_node[2] * img_f_nodes[:, 1]) + C * E ** 2 * (
                       Xfi * t_u_node[1] + xf * img_f_nodes[:, 1]) * (
                       Xfi * t_u_node[2] + xf * img_f_nodes[:, 2]) * xf * Xfi)

    m00 = MSxx - Ra * H1ski / Xf - Ra ** 3 * disxi * disxi * H3ski / Xf ** 3 - A1 * (
            img_f_nodes[:, 0] * img_f_nodes[:, 0] * H1ski / Ra ** 3 - Ra * H3ski * (
            img_f_nodes[:, 0] * disxi + img_f_nodes[:, 0] * disxi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                                 0] * img_f_nodes[:,
                                                                                      0] * Dx1 * H3ski / Ra ** 3) - A2 * Pxx
    m11 = MSyy - Ra * H1ski / Xf - Ra ** 3 * disyi * disyi * H3ski / Xf ** 3 - A1 * (
            img_f_nodes[:, 1] * img_f_nodes[:, 1] * H1ski / Ra ** 3 - Ra * H3ski * (
            img_f_nodes[:, 1] * disyi + img_f_nodes[:, 1] * disyi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                                 1] * img_f_nodes[:,
                                                                                      1] * Dx1 * H3ski / Ra ** 3) - A2 * Pyy
    m22 = MSzz - Ra * H1ski / Xf - Ra ** 3 * diszi * diszi * H3ski / Xf ** 3 - A1 * (
            img_f_nodes[:, 2] * img_f_nodes[:, 2] * H1ski / Ra ** 3 - Ra * H3ski * (
            img_f_nodes[:, 2] * diszi + img_f_nodes[:, 2] * diszi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                                 2] * img_f_nodes[:,
                                                                                      2] * Dx1 * H3ski / Ra ** 3) - A2 * Pzz

    m01 = MSxy - Ra ** 3 * disxi * disyi * H3ski / Xf ** 3 - A1 * (
            img_f_nodes[:, 0] * img_f_nodes[:, 1] * H1ski / Ra ** 3 - Ra * H3ski * (
            img_f_nodes[:, 0] * disyi + img_f_nodes[:, 1] * disxi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                                 1] * img_f_nodes[:,
                                                                                      0] * Dx1 * H3ski / Ra ** 3) - A2 * Pxy
    m02 = MSxz - Ra ** 3 * disxi * diszi * H3ski / Xf ** 3 - A1 * (
            img_f_nodes[:, 0] * img_f_nodes[:, 2] * H1ski / Ra ** 3 - Ra * H3ski * (
            img_f_nodes[:, 0] * diszi + img_f_nodes[:, 2] * disxi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                                 2] * img_f_nodes[:,
                                                                                      0] * Dx1 * H3ski / Ra ** 3) - A2 * Pxz
    m12 = MSyz - Ra ** 3 * disyi * diszi * H3ski / Xf ** 3 - A1 * (
            img_f_nodes[:, 2] * img_f_nodes[:, 1] * H1ski / Ra ** 3 - Ra * H3ski * (
            img_f_nodes[:, 2] * disyi + img_f_nodes[:, 1] * diszi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                                 1] * img_f_nodes[:,
                                                                                      2] * Dx1 * H3ski / Ra ** 3) - A2 * Pyz

    m10 = MSyx - Ra ** 3 * disxi * disyi * H3ski / Xf ** 3 - A1 * (
            img_f_nodes[:, 0] * img_f_nodes[:, 1] * H1ski / Ra ** 3 - Ra * H3ski * (
            img_f_nodes[:, 0] * disyi + img_f_nodes[:, 1] * disxi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                                 1] * img_f_nodes[:,
                                                                                      0] * Dx1 * H3ski / Ra ** 3) - A2 * Pyx
    m20 = MSzx - Ra ** 3 * disxi * diszi * H3ski / Xf ** 3 - A1 * (
            img_f_nodes[:, 0] * img_f_nodes[:, 2] * H1ski / Ra ** 3 - Ra * H3ski * (
            img_f_nodes[:, 0] * diszi + img_f_nodes[:, 2] * disxi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                                 2] * img_f_nodes[:,
                                                                                      0] * Dx1 * H3ski / Ra ** 3) - A2 * Pzx
    m21 = MSzy - Ra ** 3 * disyi * diszi * H3ski / Xf ** 3 - A1 * (
            img_f_nodes[:, 2] * img_f_nodes[:, 1] * H1ski / Ra ** 3 - Ra * H3ski * (
            img_f_nodes[:, 2] * disyi + img_f_nodes[:, 1] * diszi) / Xf ** 2 + 2 * img_f_nodes[:,
                                                                                 1] * img_f_nodes[:,
                                                                                      2] * Dx1 * H3ski / Ra ** 3) - A2 * Pzy
    return m00, m01, m02, m10, m11, m12, m20, m21, m22, i0

In [219]:
# nu, nf = 1, 2
# uth, uph = (np.random.sample((nu, 2)) * (1, 2)).T * np.pi
# # uR = Ra
# uR = np.random.sample(nu) + Ra
# t_u_node = uR * np.hstack((np.sin(uth) * np.cos(uph), np.sin(uth) * np.sin(uph), np.cos(uth)))
# fth, fph = (np.random.sample((nf, 2)) * (1, 2)).T * np.pi
# fR = np.random.sample(nf) + 1e-3
# fR = fR / fR.min() + 1e-3
# f_nodes = (fR * np.array((np.sin(fth) * np.cos(fph), np.sin(fth) * np.sin(fph), np.cos(fth)))).T

Ra = 1
t_u_node = np.array((1, 2, 3))
f_nodes = np.array((17, 13, 19)).reshape((-1, 3))
# f_nodes = np.array((7, 13, 19)).reshape((-1, 3))
# f_nodes = np.array(((7, 13, 19), (7, 13, 19), (7, 13, 19), (7, 13, 19), (7, 13, 19), (7, 13, 19)))
# f_nodes = np.array(((7, 13, 19), (17, 13, 19)))
kwargs = {'pf_sphere_image_Ra': Ra}
aaa = pf_sphere_image_petsc_mij(t_u_node, f_nodes, 0, **kwargs)

print(np.max(np.abs(np.hstack(aaa))))
print(np.max(np.abs(aaa[1] - aaa[3])))
print(np.max(np.abs(aaa[5] - aaa[7])))
print(np.max(np.abs(aaa[2] - aaa[6])))

0.04456604153652005
0.0012717426006333566
6.0559171451362616e-05
0.0019378934866821646


In [213]:
for i0 in aaa:
    print(i0)

[0.04457]
[0.00744]
[0.01076]
[0.00617]
[0.03522]
[0.00386]
[0.00882]
[0.0038]
[0.03786]
0


In [211]:
for i0 in aaa:
    print(i0)

[0.04293]
[0.00424]
[0.0061]
[0.00414]
[0.0479]
[0.01056]
[0.00591]
[0.01047]
[0.05558]
0


In [215]:
for i0 in aaa:
    print(i0)

[0.04293 0.04457]
[0.00424 0.00744]
[0.0061  0.01076]
[0.00414 0.00617]
[0.0479  0.03522]
[0.01056 0.00386]
[0.00591 0.00882]
[0.01047 0.0038 ]
[0.05558 0.03786]
0


In [216]:
print(np.max(np.abs(aaa[1] - aaa[3])))
aaa[1] - aaa[3]

0.0012717426006333566


array([9.36197e-05, 1.27174e-03])

In [220]:
10 ** 0.6

3.9810717055349722